In [1]:
import os
import random
from datetime import datetime

from faker import Faker

print("Actual path: ", os.getcwd())
os.chdir("..")
print("New path: ", os.getcwd())
from features.features_postgres import HandleDBpsql

Actual path:  /home/bdebian/Documents/Projects/tecnical_test_crystal/src/notebooks
New path:  /home/bdebian/Documents/Projects/tecnical_test_crystal/src


In [2]:
hd_pd = HandleDBpsql()
fake = Faker()

In [3]:
file_connection_parameters = (
    "/home/bdebian/Documents/Projects/tecnical_test_crystal/src/data/config/config.yaml"
)
hd_pd.check_connection(file_connection_parameters)

2024-10-02 03:23:38,229 INFO: Query de verificacion de conexion de base de datos


Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 15.3 (Debian 15.3-1.pgdg110+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 10.2.1-6) 10.2.1 20210110, 64-bit',)
Database connection closed.


In [4]:
query_insert = "/home/bdebian/Documents/Projects/tecnical_test_crystal/src/data/sql/insert_data.sql"
read_columns = "/home/bdebian/Documents/Projects/tecnical_test_crystal/src/data/sql/get_columns.sql"
query_get_table = "/home/bdebian/Documents/Projects/tecnical_test_crystal/src/data/sql/get_table.sql"
tables_to_search = ["customers", "category", "item", "ordes"]

In [5]:
def get_columns_names(query_table: str, table_name: str)->list:
    columns_to_search = {
        "name_table": "".join("'" + table_name + "'"),
        "position": "ORDINAL_POSITION",
    }

    replace_val_query = hd_pd.prepare_query_replace_value(
        sql_file=query_table, data_replace=columns_to_search
    )
    fk_cols = hd_pd.get_table(
        connection_parameters=file_connection_parameters, query=replace_val_query
    )

    return list(fk_cols.to_dict()[0].values())

def insert_data_tables(
    col_names: list, query_insert: str, table_name: str, data_insert: list
)->None:

    dict_query_ = hd_pd.fix_dict_query(
        tabla=table_name, columnas=col_names, order=None, where=" "
    )
    str_query = hd_pd.prepare_query_replace_value(
        sql_file=query_insert, data_replace=dict_query_
    )

    for data_customer in data_insert:
        hd_pd.insert_data(
            connection_parameters=file_connection_parameters,
            query=str_query,
            data=data_customer,
        )


## Insertar data de customers

In [6]:
def generate_customers(num_customers):
    customers = []
    for _ in range(num_customers):
        customer_id = fake.random_number(digits=6, fix_len=True)
        email = fake.unique.email()
        first_name = fake.first_name()
        last_name = fake.last_name()
        gender = random.choice(["M", "F", "O"])
        date_of_birth = fake.date_of_birth(minimum_age=18, maximum_age=40)
        phone = fake.random_number(digits=10, fix_len=True)
        address = fake.address()
        is_seller = fake.boolean(chance_of_getting_true=50)
        created_at = fake.date_time_between_dates(     datetime_start=datetime(2019, 1, 3, 0, 0), datetime_end=datetime(2021, 1, 1, 0, 0))()
        updated_at = created_at
        customers.append(
            (
                customer_id,
                email,
                first_name,
                last_name,
                gender,
                date_of_birth,
                phone,
                address,
                created_at,
                updated_at,
                is_seller,
            )
        )
    return customers

query para obtener nombre de las columnas

In [7]:
cols_names = get_columns_names(query_table=read_columns, table_name=tables_to_search[0])

insert_data_tables(
    col_names=cols_names,
    query_insert=query_insert,
    table_name=tables_to_search[0],
    data_insert=generate_customers(100),
)

2024-10-02 03:23:38,333 INFO: Query de insercion unica finalizada
2024-10-02 03:23:38,353 INFO: Query de insercion unica finalizada
2024-10-02 03:23:38,374 INFO: Query de insercion unica finalizada
2024-10-02 03:23:38,394 INFO: Query de insercion unica finalizada
2024-10-02 03:23:38,406 INFO: Query de insercion unica finalizada
2024-10-02 03:23:38,426 INFO: Query de insercion unica finalizada
2024-10-02 03:23:38,447 INFO: Query de insercion unica finalizada
2024-10-02 03:23:38,468 INFO: Query de insercion unica finalizada
2024-10-02 03:23:38,497 INFO: Query de insercion unica finalizada
2024-10-02 03:23:38,534 INFO: Query de insercion unica finalizada
2024-10-02 03:23:38,571 INFO: Query de insercion unica finalizada
2024-10-02 03:23:38,599 INFO: Query de insercion unica finalizada
2024-10-02 03:23:38,619 INFO: Query de insercion unica finalizada
2024-10-02 03:23:38,641 INFO: Query de insercion unica finalizada
2024-10-02 03:23:38,669 INFO: Query de insercion unica finalizada
2024-10-02

## Insertar data de categorias

In [8]:
def generate_categories(num_categories):
    categories = []
    for i in range(1, num_categories + 1):
        category_id = i
        name = fake.unique.word()
        parent_id = random.choice([None] + list(range(1, i)))
        level = 0 if parent_id is None else 1
        path = (
            f"/{category_id}"
            if parent_id is None
            else f"/{parent_id}/{category_id}"
        )
        created_at = fake.date_time_between_dates(     datetime_start=datetime(2019, 1, 3, 0, 0), datetime_end=datetime(2021, 1, 1, 0, 0))()
        updated_at = created_at
        categories.append(
            (category_id, name, parent_id, level, path, created_at, updated_at)
        )
    return categories

In [9]:
cols_names = get_columns_names(query_table=read_columns, table_name=tables_to_search[1])
insert_data_tables(
    col_names=cols_names,
    query_insert=query_insert,
    table_name=tables_to_search[1],
    data_insert=generate_categories(20),
)

2024-10-02 03:23:41,439 ERROR: Fallo de insercion de la query
2024-10-02 03:23:41,440 ERROR: duplicate key value violates unique constraint "category_pkey"
DETAIL:  Key (category_id)=(1) already exists.

2024-10-02 03:23:41,444 ERROR: Fallo de insercion de la query
2024-10-02 03:23:41,444 ERROR: duplicate key value violates unique constraint "category_pkey"
DETAIL:  Key (category_id)=(2) already exists.

2024-10-02 03:23:41,448 ERROR: Fallo de insercion de la query
2024-10-02 03:23:41,449 ERROR: duplicate key value violates unique constraint "category_pkey"
DETAIL:  Key (category_id)=(3) already exists.

2024-10-02 03:23:41,453 ERROR: Fallo de insercion de la query
2024-10-02 03:23:41,453 ERROR: duplicate key value violates unique constraint "category_pkey"
DETAIL:  Key (category_id)=(4) already exists.

2024-10-02 03:23:41,457 ERROR: Fallo de insercion de la query
2024-10-02 03:23:41,458 ERROR: duplicate key value violates unique constraint "category_pkey"
DETAIL:  Key (category_id)=(

2024-10-02 03:23:41,463 ERROR: Fallo de insercion de la query
2024-10-02 03:23:41,463 ERROR: duplicate key value violates unique constraint "category_pkey"
DETAIL:  Key (category_id)=(6) already exists.

2024-10-02 03:23:41,467 ERROR: Fallo de insercion de la query
2024-10-02 03:23:41,468 ERROR: duplicate key value violates unique constraint "category_pkey"
DETAIL:  Key (category_id)=(7) already exists.

2024-10-02 03:23:41,473 ERROR: Fallo de insercion de la query
2024-10-02 03:23:41,474 ERROR: duplicate key value violates unique constraint "category_pkey"
DETAIL:  Key (category_id)=(8) already exists.

2024-10-02 03:23:41,478 ERROR: Fallo de insercion de la query
2024-10-02 03:23:41,479 ERROR: duplicate key value violates unique constraint "category_pkey"
DETAIL:  Key (category_id)=(9) already exists.

2024-10-02 03:23:41,484 ERROR: Fallo de insercion de la query
2024-10-02 03:23:41,484 ERROR: duplicate key value violates unique constraint "category_pkey"
DETAIL:  Key (category_id)=(

## Insertar data de Items

obtener los datos de la tabla category

In [10]:
table_name =tables_to_search[1]
cols_names = get_columns_names(query_table=read_columns, table_name=table_name)

dict_query_ = hd_pd.fix_dict_query(
    tabla=table_name,
    columnas=cols_names,
    order="category_id",
    where='"category_id" > 0',
)
replace_val_query = hd_pd.prepare_query_replace_value(
    sql_file=query_get_table, data_replace=dict_query_
)
table_ctg = hd_pd.get_table(
    connection_parameters=file_connection_parameters, query=replace_val_query
)
table_ctg.rename(dict(zip(table_ctg.columns, cols_names)), axis=1, inplace=True)
display(table_ctg)

,category_id,name,parent_id,level,path,created_at,updated_at
0,1,medical,NaN,0,/1,2020-02-08 04:29:29.804666,2020-02-08 04:29:29.804666
1,2,majority,NaN,0,/2,2021-06-09 12:27:49.645509,2021-06-09 12:27:49.645509
2,3,memory,NaN,0,/3,2023-06-18 03:46:01.482970,2023-06-18 03:46:01.482970
3,4,morning,3.0,1,/3/4,2020-07-24 21:01:36.666461,2020-07-24 21:01:36.666461
4,5,thing,2.0,1,/2/5,2023-02-03 17:32:45.246175,2023-02-03 17:32:45.246175
5,6,body,2.0,1,/2/6,2023-11-27 19:45:26.659929,2023-11-27 19:45:26.659929
6,7,firm,6.0,1,/6/7,2020-04-28 11:17:52.981209,2020-04-28 11:17:52.981209
7,8,age,3.0,1,/3/8,2020-08-14 15:25:08.129192,2020-08-14 15:25:08.129192
8,9,garden,5.0,1,/5/9,2020-01-07 02:20:53.146829,2020-01-07 02:20:53.146829
9,10,I,3.0,1,/3/10,2021-02-18 01:15:56.795070,2021-02-18 01:15:56.795070


obtener los datos de la tabla customer

In [11]:
table_name = tables_to_search[0]
cols_names = get_columns_names(query_table=read_columns, table_name=table_name)

dict_query_ = hd_pd.fix_dict_query(
    tabla=table_name,
    columnas=cols_names,
    order="customer_id",
    where='"customer_id" > 0',
)
replace_val_query = hd_pd.prepare_query_replace_value(
    sql_file=query_get_table, data_replace=dict_query_
)
table_customer = hd_pd.get_table(
    connection_parameters=file_connection_parameters, query=replace_val_query
)
table_customer.rename(
    dict(zip(table_customer.columns, cols_names)), axis=1, inplace=True
)
display(table_customer)

,customer_id,email,first_name,last_name,gender,date_of_birth,phone,address,created_at,updated_at,is_seller
0,101245,rcastillo@example.com,Emily,Levine,O,1978-05-28,2155124729,"905 Benjamin Causeway\nDennismouth, AK 81079",2012-02-20 12:34:00.091808,2012-02-20 12:34:00.091808,True
1,104045,lynnjonathan@example.org,Sarah,Mccoy,O,1967-10-15,5494150970,"04535 Destiny Oval\nBrianachester, AR 57804",2021-08-30 11:24:06.101788,2021-08-30 11:24:06.101788,False
2,109201,townsendkristina@example.net,Michele,Henderson,F,1977-08-16,7872733422,"873 Charles Mills\nPort Theresaland, CO 29545",2021-03-15 09:26:26.039617,2021-03-15 09:26:26.039617,True
3,112559,frose@example.com,Billy,Baker,O,1991-04-25,2230504188,"0084 Andrew Lakes Suite 482\nCunninghamburgh, ...",2020-01-11 21:57:56.585249,2020-01-11 21:57:56.585249,True
4,123559,ericcortez@example.com,Austin,Bonilla,O,1979-05-04,8987585738,"7460 Angela Cliff Suite 484\nPort Jeffreyfurt,...",2013-04-09 09:18:33.518269,2013-04-09 09:18:33.518269,True
...,...,...,...,...,...,...,...,...,...,...,...
294,984720,tkim@example.com,Micheal,Bush,M,1988-05-01,6154358139,"PSC 5449, Box 4813\nAPO AP 84830",2010-10-13 06:07:10.783278,2010-10-13 06:07:10.783278,True
295,985823,greenadam@example.org,Ann,Dixon,F,2000-10-24,7536656144,"26494 Rose Mountain\nSherryfort, NJ 96648",2003-08-21 01:06:04.427836,2003-08-21 01:06:04.427836,True
296,988940,wilsonjeffrey@example.net,Erica,Delgado,M,1987-08-15,4572420395,"54534 Tracy Islands Suite 426\nEast Gary, FM 5...",2022-10-14 01:15:21.622746,2022-10-14 01:15:21.622746,False
297,993033,leerebecca@example.net,Joshua,Alexander,O,1967-11-03,7118122699,"98160 Livingston Ville\nPowellside, OR 41068",2024-02-28 07:26:51.306510,2024-02-28 07:26:51.306510,False


In [12]:
table_name = "statusitem"
cols_names = get_columns_names(query_table=read_columns, table_name=table_name)

insert_data_tables(
    col_names=cols_names,
    query_insert=query_insert,
    table_name=table_name,
    data_insert=[(1, "active"), (2, "inactive"), (3, "deleted"),],
)

2024-10-02 03:23:41,620 ERROR: Fallo de insercion de la query
2024-10-02 03:23:41,620 ERROR: duplicate key value violates unique constraint "statusitem_pkey"
DETAIL:  Key (status_item_id)=(1) already exists.

2024-10-02 03:23:41,624 ERROR: Fallo de insercion de la query
2024-10-02 03:23:41,625 ERROR: duplicate key value violates unique constraint "statusitem_pkey"
DETAIL:  Key (status_item_id)=(2) already exists.

2024-10-02 03:23:41,629 ERROR: Fallo de insercion de la query
2024-10-02 03:23:41,629 ERROR: duplicate key value violates unique constraint "statusitem_pkey"
DETAIL:  Key (status_item_id)=(3) already exists.



In [33]:
import pandas as pd
# Función para generar items
def generate_items(num_items, table_customer: pd.DataFrame, table_ctg:pd.DataFrame):
    items = []
    for _ in range(num_items):
        item_id = fake.random_number(digits=6, fix_len=True)
        seller_id = int(
            table_customer.iloc[random.randint(1, 99)]["customer_id"]
        )  # Suponiendo que ya se han generado 100 usuarios
        category_id = int(
            table_ctg.iloc[random.randint(0, 19)]["category_id"]
        )  # Suponiendo que ya se han generado 20 categorías
        name = fake.word()
        description = fake.text(max_nb_chars=200)
        price = round(random.uniform(5.0, 500.0), 2)
        # status = random.choice(["active", "inactive", "deleted"])
        status = random.choice(range(1, 4))
        deletion_date = (
            fake.date_time_between_dates(datetime_start=datetime(2019, 1, 3, 0, 0), datetime_end=datetime(2021, 1, 1, 0, 0)) if status == 3 else None
        )
        created_at = fake.date_time_between_dates(datetime_start=datetime(2019, 1, 3, 0, 0), datetime_end=datetime(2021, 1, 1, 0, 0))
        updated_at = created_at
        items.append(
            (
                item_id,
                seller_id,
                category_id,
                name,
                description,
                price,
                status,
                deletion_date,
                created_at,
                updated_at,
            )
        )
    return items

In [34]:
cols_names = get_columns_names(query_table=read_columns, table_name=tables_to_search[2])
insert_data_tables(
    col_names=cols_names,
    query_insert=query_insert,
    table_name=tables_to_search[2],
    data_insert=generate_items(50, table_customer=table_customer, table_ctg=table_ctg),
)

2024-10-02 03:34:35,065 INFO: Query de insercion unica finalizada
2024-10-02 03:34:35,102 INFO: Query de insercion unica finalizada
2024-10-02 03:34:35,122 INFO: Query de insercion unica finalizada
2024-10-02 03:34:35,144 INFO: Query de insercion unica finalizada
2024-10-02 03:34:35,164 INFO: Query de insercion unica finalizada
2024-10-02 03:34:35,184 INFO: Query de insercion unica finalizada
2024-10-02 03:34:35,213 INFO: Query de insercion unica finalizada
2024-10-02 03:34:35,233 INFO: Query de insercion unica finalizada
2024-10-02 03:34:35,262 INFO: Query de insercion unica finalizada
2024-10-02 03:34:35,291 INFO: Query de insercion unica finalizada
2024-10-02 03:34:35,327 INFO: Query de insercion unica finalizada
2024-10-02 03:34:35,348 INFO: Query de insercion unica finalizada
2024-10-02 03:34:35,385 INFO: Query de insercion unica finalizada
2024-10-02 03:34:35,406 INFO: Query de insercion unica finalizada
2024-10-02 03:34:35,446 INFO: Query de insercion unica finalizada
2024-10-02

## Insertar data de Orders

Rellenar status order

In [15]:
table_name = "statusord"
cols_names = get_columns_names(query_table=read_columns, table_name=table_name)

insert_data_tables(
    col_names=cols_names,
    query_insert=query_insert,
    table_name=table_name,
    data_insert=[
        (1, "pending"),
        (2, "completed"),
        (3, "shipped"),
        (4, "cancelled"),
    ]
)

2024-10-02 03:23:43,112 ERROR: Fallo de insercion de la query
2024-10-02 03:23:43,113 ERROR: duplicate key value violates unique constraint "statusord_pkey"
DETAIL:  Key (status_ord_id)=(1) already exists.

2024-10-02 03:23:43,116 ERROR: Fallo de insercion de la query
2024-10-02 03:23:43,117 ERROR: duplicate key value violates unique constraint "statusord_pkey"
DETAIL:  Key (status_ord_id)=(2) already exists.



2024-10-02 03:23:43,122 ERROR: Fallo de insercion de la query
2024-10-02 03:23:43,122 ERROR: duplicate key value violates unique constraint "statusord_pkey"
DETAIL:  Key (status_ord_id)=(3) already exists.

2024-10-02 03:23:43,129 ERROR: Fallo de insercion de la query
2024-10-02 03:23:43,129 ERROR: duplicate key value violates unique constraint "statusord_pkey"
DETAIL:  Key (status_ord_id)=(4) already exists.



obtener los datos de la tabla de items

In [16]:
table_name = tables_to_search[2]
cols_names = get_columns_names(query_table=read_columns, table_name=table_name)

dict_query_ = hd_pd.fix_dict_query(
    tabla=table_name,
    columnas=cols_names,
    order="item_id",
    # where='"status_item" = 1',
    where='"status_item" > 0',
)
replace_val_query = hd_pd.prepare_query_replace_value(
    sql_file=query_get_table, data_replace=dict_query_
)
table_item = hd_pd.get_table(
    connection_parameters=file_connection_parameters, query=replace_val_query
)
table_item.rename(
    dict(zip(table_item.columns, cols_names)), axis=1, inplace=True
)
display(table_item)

,item_id,seller_id,category_id,name,description,price,status_item,deletion_date,created_at,updated_at
0,106617,303133,10,president,Itself site treatment hospital move lawyer rec...,185.71,2,NaT,2021-02-06 17:52:00.475863,2021-02-06 17:52:00.475863
1,124023,167335,17,series,Writer admit war my sort scientist. Find five ...,100.52,1,NaT,2002-04-21 04:15:15.329533,2002-04-21 04:15:15.329533
2,147333,200728,12,road,Offer capital unit right speak difference. Tou...,39.59,2,NaT,2010-04-06 23:08:25.879231,2010-04-06 23:08:25.879231
3,148795,144216,15,ability,Money fund walk quality those toward way. Fina...,142.10,1,NaT,2020-04-06 19:00:56.070056,2020-04-06 19:00:56.070056
4,160875,320129,3,most,Road civil within reveal.\nOutside politics wh...,332.67,1,NaT,2010-02-07 23:40:21.144835,2010-02-07 23:40:21.144835
...,...,...,...,...,...,...,...,...,...,...
145,976477,181150,17,process,Sell if strategy five often senior. Rich turn ...,88.48,2,NaT,2020-11-09 08:23:06.268674,2020-11-09 08:23:06.268674
146,976779,167335,15,dream,Avoid total care push oil daughter change. Con...,327.72,1,NaT,2005-08-23 21:56:51.172608,2005-08-23 21:56:51.172608
147,983752,153198,9,church,Green lead gun. Statement everybody developmen...,69.69,2,NaT,2021-04-17 04:51:58.497916,2021-04-17 04:51:58.497916
148,986120,365404,4,cell,Interest artist above else address lose. Toget...,441.81,3,2022-07-10 05:44:52.258137,2020-08-12 23:48:51.506771,2020-08-12 23:48:51.506771


In [35]:
def generate_orders(num_orders: int, table_customer: pd.DataFrame, table_item:pd.DataFrame):
    orders = []
    for _ in range(num_orders):
        ord_id = fake.random_number(digits=6, fix_len=True)
        buyer_id = int(table_customer.iloc[random.randint(1, 99)]["customer_id"])
        item_id = int(table_item.iloc[random.randint(1, 48)]['item_id'])
        quantity = random.randint(1, 10)
        total_price = round(quantity * random.uniform(5.0, 500.0), 2)
        ord_date = fake.date_time_between_dates(datetime_start=datetime(2019, 1, 3, 0, 0), datetime_end=datetime(2021, 1, 1, 0, 0))
        status_ord = random.randint(1, 4)
        created_at = ord_date
        updated_at = created_at

        orders.append(
            (
                ord_id,
                buyer_id,
                item_id,
                quantity,
                total_price,
                ord_date,
                status_ord,
                created_at,
                updated_at,
            )
        )
    return orders

In [37]:
table_name = tables_to_search[3]
cols_names = get_columns_names(query_table=read_columns, table_name=table_name)

insert_data_tables(
    col_names=cols_names,
    query_insert=query_insert,
    table_name=table_name,
    data_insert=generate_orders(1000, table_customer, table_item),
)

2024-10-02 03:55:28,416 INFO: Query de insercion unica finalizada
2024-10-02 03:55:28,437 INFO: Query de insercion unica finalizada
2024-10-02 03:55:28,457 INFO: Query de insercion unica finalizada
2024-10-02 03:55:28,477 INFO: Query de insercion unica finalizada
2024-10-02 03:55:28,498 INFO: Query de insercion unica finalizada
2024-10-02 03:55:28,518 INFO: Query de insercion unica finalizada
2024-10-02 03:55:28,547 INFO: Query de insercion unica finalizada
2024-10-02 03:55:28,568 INFO: Query de insercion unica finalizada
2024-10-02 03:55:28,604 INFO: Query de insercion unica finalizada
2024-10-02 03:55:28,642 INFO: Query de insercion unica finalizada
2024-10-02 03:55:28,679 INFO: Query de insercion unica finalizada
2024-10-02 03:55:28,716 INFO: Query de insercion unica finalizada
2024-10-02 03:55:28,753 INFO: Query de insercion unica finalizada
2024-10-02 03:55:28,781 INFO: Query de insercion unica finalizada
2024-10-02 03:55:28,802 INFO: Query de insercion unica finalizada
2024-10-02